# Laboratorium 4 - rekomendacje dla portali informacyjnych

## Przygotowanie

 * pobierz i wypakuj dataset: https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
   * więcej możesz poczytać tutaj: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-microsoft-news
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab4`
 * zainstaluj potrzebne biblioteki:
 `pip install nltk sklearn`

## Część 1. - przygotowanie danych

In [6]:
# importujemy wszystkie potrzebne pakiety

import codecs
from collections import defaultdict # mozesz uzyc zamiast zwyklego slownika, rozwaz wplyw na czas obliczen
import math
import re
from string import punctuation

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# mozesz uzyc do obliczania najbardziej podobnych tekstow zamiast liczenia "na piechote"
# ale pamietaj o dostosowaniu formatu danych
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package stopwords to /Users/mateu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# definiujemy potrzebne zmienne

PATH = './MINDsmall_train'
STOPWORDS = set(stopwords.words('english'))

In [8]:
# wczytujemy metadane artykułów

def parse_news_entry(entry):
    news_id, category, subcategory, title, abstract = entry.split('\t')[:5]
    return {
        'news_id': news_id,
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'abstract': abstract
    }

def get_news_metadata():
    with codecs.open(f'{PATH}/news.tsv', 'r', 'UTF-8') as f:
        raw = [x for x in f.read().split('\n') if x]
        parsed_entries = [parse_news_entry(entry) for entry in raw]
        return {x['news_id']: x for x in parsed_entries}

news = get_news_metadata()
news_ids = sorted(list(news.keys()))
news_indices = {x[1]: x[0] for x in enumerate(news_ids)}
print(len(news))

51282


## Część 2. - TF-IDF

In [11]:
# normalizujemy teksty na potrzeby dalszego przetwarzania

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in STOPWORDS]
    return text

def stem_texts(corpus):
    stemmer = SnowballStemmer('english')
    return [[stemmer.stem(word) for word in preprocess_text(text)] for text in corpus]

texts = [news[news_id]['abstract'] for news_id in news_ids]
stemmed_texts = stem_texts(texts)

In [12]:
# porownajmy teksty przed i po przetworzeniu
print(texts[2] + '\n')
print(' '.join(stemmed_texts[2]))

"I think we have a really good team, and a team that can really do some special, good things because that group is very close in there." - Brian Schmetzer

think realli good team team realli special good thing group close brian schmetzer


In [13]:
# tworzymy liste wszystkich slow w korpusie

def get_all_words_sorted(corpus):
    # generujemy posortowana alfabetycznie liste wszystkich slow (tokenow)
    return sorted(list(set([word for text in corpus for word in text])))

wordlist = get_all_words_sorted(stemmed_texts)
word_indices = {x[1]: x[0] for x in enumerate(wordlist)}
print(len(wordlist))

41691


In [24]:
# obliczamy liczbe tekstow, w ktorych wystapilo kazde ze slow
# pamietaj, ze jesli slowo wystapilo w danym tekscie wielokrotnie, to liczymy je tylko raz

def get_document_frequencies(corpus):
    result = defaultdict(int)
    for text in corpus:
        for word in set(text):
            result[word] += 1
    return result

document_frequency = get_document_frequencies(stemmed_texts)

In [25]:
document_frequency['news']

1639

In [26]:
# obliczamy liczbe wystapien kazdego slowa w kazdym tekscie

def get_term_frequencies(corpus, news_indices):
    term_frequency = defaultdict(lambda: defaultdict(int))
    for news_id, text in zip(news_indices, corpus):
        for word in text:
            term_frequency[news_id][word] += 1
    return term_frequency

term_frequency = get_term_frequencies(stemmed_texts, news_indices)

In [27]:
# sprawdzmy wyniki

term_frequency[news_ids[2]]

defaultdict(int,
            {'think': 1,
             'realli': 2,
             'good': 2,
             'team': 2,
             'special': 1,
             'thing': 1,
             'group': 1,
             'close': 1,
             'brian': 1,
             'schmetzer': 1})

In [28]:
# obliczamy metryke tf_idf

def calculate_tf_idf(term_frequency, document_frequency, corpus_size):
    tf_idf = defaultdict(lambda: defaultdict(float))
    for news_id, text in term_frequency.items():
        for word, frequency in text.items():
            tf_idf[news_id][word] = frequency * math.log(corpus_size / document_frequency[word])
    return tf_idf

tf_idf = calculate_tf_idf(term_frequency, document_frequency, len(news_ids))

In [29]:
# sprawdzmy wyniki

tf_idf[news_ids[2]]

defaultdict(float,
            {'think': 4.360459856758821,
             'realli': 9.707261090572182,
             'good': 8.085400658139525,
             'team': 6.157356388748834,
             'special': 4.765161897298483,
             'thing': 4.110503432421125,
             'group': 4.261685870235309,
             'close': 3.9244235881453897,
             'brian': 5.624739267315748,
             'schmetzer': 9.458800731274183})

## Część 3. - Podobieństwo tekstów

In [49]:
# obliczmy odleglosc miedzy dwoma artykulami
# przetestuj rozne metryki odleglosci i wybierz najlepsza

def calculate_distance(tf_idf, id1, id2, metric='euclidean'):
    if metric == 'euclidean':
        return math.sqrt(sum([(tf_idf[id1][word] - tf_idf[id2][word]) ** 2 for word in tf_idf[id1] if word in tf_idf[id2]]))
    elif metric == 'cosine':
        return sum([tf_idf[id1][word] * tf_idf[id2][word] for word in tf_idf[id1] if word in tf_idf[id2]])
    elif metric == 'taxicab':
        return sum([abs(tf_idf[id1][word] - tf_idf[id2][word]) for word in tf_idf[id1] if word in tf_idf[id2]])
    elif metric == 'minkowski':
        return sum([abs(tf_idf[id1][word] - tf_idf[id2][word]) ** 3 for word in tf_idf[id1] if word in tf_idf[id2]]) ** (1 / 3)
    elif metric == 'chebyshev':
        return max([abs(tf_idf[id1][word] - tf_idf[id2][word]) for word in tf_idf[id1] if word in tf_idf[id2]])
    elif metric == 'jaccard':
        return sum([min(tf_idf[id1][word], tf_idf[id2][word]) for word in tf_idf[id1] if word in tf_idf[id2]]) / sum([max(tf_idf[id1][word], tf_idf[id2][word]) for word in tf_idf[id1] if word in tf_idf[id2]])
    elif metric == 'dice':
        return 2 * sum([min(tf_idf[id1][word], tf_idf[id2][word]) for word in tf_idf[id1] if word in tf_idf[id2]]) / (sum([tf_idf[id1][word] for word in tf_idf[id1] if word in tf_idf[id2]]) + sum([tf_idf[id2][word] for word in tf_idf[id1] if word in tf_idf[id2]]))
    raise ValueError('Unknown metric')

calculate_distance(tf_idf, news_ids[2], news_ids[1], 'euclidean')

19.554262136437274

In [50]:
# Print a table with results for all metrics
metrics = 'euclidean', 'cosine', 'taxicab', 'minkowski', 'chebyshev', 'jaccard', 'dice'
results = [calculate_distance(tf_idf, news_ids[2], news_ids[1], metric) for metric in metrics]

# Create a string printing the table
results_string = ''
for metric, result in zip(metrics, results):
  results_string += f'{metric}: {result}\n'

# Print the table
print(results_string)

euclidean: 19.554262136437274
cosine: 18.956518849033042
taxicab: 57.37711458653518
minkowski: 14.252398792095919
chebyshev: 9.707261090572182
jaccard: 0.050924453270036106
dice: 0.09691363277651513



In [51]:
# wyznaczmy k najpodobniejszych tekstow do danego
# pamietaj o odpowiedniej kolejnosci sortowania w zaleznosci od wykorzystanej metryki
# pamietaj, zeby wsrod podobnych tekstow nie bylo danego

def get_k_most_similar_news(tf_idf, n_id, k, metric='euclidean'):
    if metric == 'euclidean':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric))[:k]
    elif metric == 'cosine':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric), reverse=True)[:k]
    elif metric == 'taxicab':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric))[:k]
    elif metric == 'minkowski':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric))[:k]
    elif metric == 'chebyshev':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric))[:k]
    elif metric == 'jaccard':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric), reverse=True)[:k]
    elif metric == 'dice':
        return sorted([id for id in tf_idf if id != n_id], key=lambda x: calculate_distance(tf_idf, n_id, x, metric), reverse=True)[:k]    

def print_k_most_similar_news(tf_idf, n_id, k, corpus, news_indices, metric='euclidean'):
    similar = get_k_most_similar_news(tf_idf, n_id, k, metric)
    print(f'id: {n_id}, text: {corpus[news_indices[n_id]]}')
    print(f'\n{k} most similar:')
    for s_id in similar:
        print(f'\nid: {s_id}, text: {corpus[news_indices[s_id]]}')

print_k_most_similar_news(tf_idf, news_ids[42337], 5, texts, news_indices, 'cosine')

id: N58544, text: A MAN claims he has created a car that might solve the world's traffic congestion problems. Rick Woodbury from Spokane, Washington USA, is the president, founder and sole employee of 'Commuter Cars.' The carmaker's flagship model is the 2005 super slim two-seater Tango T600, a high-performance electric car that preceded Tesla. Rick told BTV: "I started this company 21 years ago   it was based on an idea that I came up with in 1982." He was inspired by the shocking traffic congestion he had to face on a daily basis. "I used to drive a Porsche from Beverly Hills to Hermosa Beach every day and the traffic was horrendous," explained Rick. What really made Rick think about a solution was the fact that in most of the cars he would see in his commute were occupied by lone drivers. "I noticed that everybody around me was a single occupant in a car, taking up the whole lane," Rick said. Living and working in Los Angeles also helped inspire Rick's unique creation. "I thought, e